# In Class Exercise #2

Professor: Bolton
Class: IST 736

Group Members: ___ 



## Instructions: 

This exercise is designed to prep you with skills needed to complete your assignment this week. Complete the following questions and submit a link to your completed notebook file as instructed. You may work in groups.

## Schedule:

While in groups, please appropriate your time as follows: 

     - Q & A (20 minutes)
     Discuss in your group any lingering questions from the previous week, or discussion any questions / challenges encountered while reviewing this weeks material. Attempt to share and problem solve with your groupmates. Your instructor will drop into break out groups during this time to assist with the discussion and Q&A. Feel free to discussion concepts, coding, assignments, python tools, best practices, packages, debugging strategies, issues, tips, hints, etc. Use this time wisely! Share info and share tips!

    - In-Class Exercise (50 minutes)
    Complete the questions and exercises found within this notbook file with your group. Each group need to create 1 file but each individual will submit a link to that file within the git classroom. You can also render the notebook and upload copies if you prefer. Be sure to include *ALL GROUP MEMBERS NAMES* in the file at the top.
    
    The goal of the in-class exercises is to prep you with the concepts and skills needed to successfully complete that corresponding weeks HW assignment. Take full advantage of this time and work together to maximize the learning and understanding. Your professor will stop by all of the groups to help with any questions.
    
    - Peer Feedback (20 minutes)
    Each group will review another groups submission. This feedback should include 
            -- positive feedback, eg, what was awesome!
            -- constructive criticism, eg, how might you improve
            -- tips and hints, eg, coding style, other packages, etc
    
    - Notes:
    If you complete this schedule early you are welcome to leave early or feel free to stick around :) 
    

## Resources:

Resources to help complete this assignment:

    - Check python docs for the corresponding python packages used.
    - https://github.com/boltonvandy/IST736repo
    

## Professor Feedback:

Most professor feedback will be provided asynchronously via the git classroom after class. During class your professor will be focused on observing participation of students, facilitating the class, answering questions, and tracking bugs (ugh ... ). It is important to understand this as there will be multiple groups and tasks. Your professor will work to assist as soon as practical if you have questions or encounter obstacles, but please be patient :). 

If there are any lingering, unsolved questions at the end of class, please document them and include them in the submission. Your professor will address any such questions within the feedback. 

Your professor will also offer extensive feedback about coding best practices, helpful hints, python tips, useful python packages, resources, and more!

## Grading:

Grades will be based on participation and effort of all 3 major components: discussion, in-class exercise, and peer feedback. 

In [1]:
!pip install --upgrade pixiedust

In [2]:
# Import packages for data cleaning, 
# viz and some sentiment analysis for fun :)

import pandas as pd
import nltk
nltk.download('vader_lexicon')
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pixiedust 



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jerem\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Pixiedust database opened successfully


# Introduction.

Lets load some twitter data. Our goal will be to "clean" it up! We will investigate using stemmers and stopwords! The base code for loading is provided below. Feel free to use a different dataset if you prefer. 

In [3]:
# Data files to be used. Political Twitter Data.
#sourceDir = ""
tweetsFile = "ExtractedTweets.csv"
twitterHandles = "TwitterHandles.csv"

In [4]:
justTweetsDf = pd.read_csv(tweetsFile)
justHandlesDf = pd.read_csv(twitterHandles)

In [5]:
# Check to confirm data loaded correctly
justTweetsDf.head()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...


In [6]:
# Begin to investigate data. Find num of Dem and Rep

print(len(justTweetsDf[justTweetsDf['Party']=='Democrat']))
print()
print(len(justTweetsDf[justTweetsDf['Party']=='Republican']))
print()
print(len(justTweetsDf))

42068

44392

86460


In [7]:
print(justHandlesDf.shape)
justHandlesDf = justHandlesDf.drop_duplicates(subset='Handle')
print(justHandlesDf.shape)

(48, 4)
(40, 4)


In [8]:
justHandlesDf.head()

,Party,Name,Handle,AvatarURL
0,Democrat,US Rep. Darren Soto,RepDarrenSoto,https://pbs.twimg.com/profile_images/824454906...
1,Democrat,Rep. Jacky Rosen,RepJackyRosen,https://pbs.twimg.com/profile_images/837772241...
2,Democrat,US Rep. Al Lawson Jr,RepAlLawsonJr,https://pbs.twimg.com/profile_images/818493713...
3,Democrat,Adriano Espaillat,RepEspaillat,https://pbs.twimg.com/profile_images/827580972...
8,Democrat,Rep. Blunt Rochester,RepBRochester,https://pbs.twimg.com/profile_images/912673706...


In [9]:
tweetsDf = justTweetsDf.merge(copy=True, right=justHandlesDf, on='Handle', how='left')
tweetsDf = tweetsDf.drop(columns=['Party_y','AvatarURL'])
tweetsDf = tweetsDf.rename(columns={'Party_x':'Party'})

In [10]:
len(tweetsDf['Handle'].unique())

433

In [11]:
# Clean some of the text data. EG get rid of "RT"

tweetsDf['IsRetweet'] = tweetsDf['Tweet'].str.startswith('RT')
tweetsDf['Tweet'] = tweetsDf['Tweet'].str.replace('RT ', '').str.strip()

In [12]:
# Quartile Analysis on wordcount

tweetsDf['WordCount'] = tweetsDf['Tweet'].str.count("\S\s+\S")+1
tweetsDf['WordCount'].describe()

count    86460.000000
mean        17.823225
std          4.253023
min          1.000000
25%         16.000000
50%         19.000000
75%         21.000000
max         31.000000
Name: WordCount, dtype: float64

In [13]:
# More EDA!! Peak at data and compute mean word counts 

print(tweetsDf['Tweet'][0])
print(tweetsDf['WordCount'][0])

print(tweetsDf[tweetsDf['Party']=='Democrat']['WordCount'].mean())
print(tweetsDf[tweetsDf['Party']=='Republican']['WordCount'].mean())

print(len(tweetsDf[tweetsDf['Party']=='Democrat']))
print(len(tweetsDf[tweetsDf['Party']=='Republican']))


Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L
17
18.045878102120376
17.612227428365472
42068
44392


# Question #1

Tokenize Tweets using tweet tokenizer. See TweetTokenizer docs for syntax. 

In [14]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
tt = TweetTokenizer()

# Tokenize your tweets dataframe her and confirm tokenizer worked :)

# Question #2

Create a wordcloud to identify high frequency words. What did you find? Are there any words that should be added to the stopword list? Why?

In [15]:
#  Create a wordcloud to help viz the text data
# Great step for textual EDA
# Don;t forget to remove stop words!!

import wordcloud

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# <insert code here>

# Create wordcloud. See wordcloud docs for syntax. 

stopWords = stopwords.words('english')
stopWords.append('')

# append stopwords to list
# create new word cloud


# Question #3. 

How many words did you remove in total? What was the word count before and after your stopword analyses? Show your code and answer.

# Question #4

Why are stemmers used in general? Use snowballstemmer or other to stem words. What notable changes do you observe? 

In [16]:
# Apply stemmer and or lemmatizer!!

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

# iterate through tweets and combine into a list. Then apply stemmer.
# Visualize and or quantify results to confirm stemmer is working as desired. 

# Question #5

What words are most frequent for Democrat Tweets? How about Republican Tweets? What is the frequency for the top 5 most frequent words? 

In [17]:
#<insert code and answer here>